In [80]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [95]:
news = pd.read_csv('/Users/mackdelany/Documents/DSR Notebooks/NLP-stock-price-prediction/data/train/RedditNews_train.csv')
labels = pd.read_csv('/Users/mackdelany/Documents/DSR Notebooks/NLP-stock-price-prediction/data/train/Combined_News_DJIA_train.csv')





In [82]:
print(news.shape)

(58908, 2)


In [83]:
news.dropna(inplace=True)

In [100]:
news.head()

,Date,News
0,2008-06-08,"b'Marriage, they said, was reduced to the stat..."
1,2008-06-08,b'Nim Chimpsky: The tragedy of the chimp who t...
2,2008-06-08,"b""Canada: Beware slippery slope' to censorship..."
3,2008-06-08,b'EU Vice-President Luisa Morgantini and the I...
4,2008-06-08,NaN


In [85]:
news['News'] = news['News'].apply(lambda x: x.lower())

In [86]:
news = news.groupby(['Date'])['News'].apply(lambda x: ', '.join(x)).reset_index()

In [87]:
corpus = news['News'].values

In [88]:
transformer = TfidfVectorizer()

In [89]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=200)

In [90]:
news_vectors = vectorizer.fit_transform(corpus).toarray()

In [107]:
vectorizer.get_feature_names()

['000',
 '10',
 '100',
 '20',
 'accused',
 'afghan',
 'afghanistan',
 'africa',
 'aid',
 'air',
 'al',
 'american',
 'amp',
 'anti',
 'arab',
 'army',
 'arrested',
 'attack',
 'attacks',
 'australia',
 'australian',
 'ban',
 'bank',
 'bbc',
 'billion',
 'border',
 'brazil',
 'britain',
 'british',
 'calls',
 'canada',
 'canadian',
 'case',
 'chief',
 'child',
 'children',
 'china',
 'chinese',
 'city',
 'claims',
 'countries',
 'country',
 'court',
 'crisis',
 'day',
 'days',
 'dead',
 'death',
 'drug',
 'east',
 'egypt',
 'egyptian',
 'election',
 'end',
 'eu',
 'europe',
 'european',
 'face',
 'food',
 'forces',
 'foreign',
 'france',
 'free',
 'french',
 'gay',
 'gaza',
 'german',
 'germany',
 'girl',
 'global',
 'government',
 'group',
 'hamas',
 'help',
 'high',
 'hit',
 'home',
 'human',
 'illegal',
 'including',
 'india',
 'indian',
 'international',
 'internet',
 'iran',
 'iranian',
 'iraq',
 'islamic',
 'israel',
 'israeli',
 'japan',
 'just',
 'kill',
 'killed',
 'killing',
 

In [108]:
vectorizer.vocabulary_

{'said': 152,
 'women': 195,
 'canada': 30,
 'eu': 54,
 'president': 137,
 'international': 82,
 'army': 15,
 'anti': 13,
 'drug': 48,
 'shot': 160,
 'dead': 46,
 'news': 115,
 'pakistan': 124,
 'face': 57,
 'make': 102,
 'people': 131,
 'britain': 27,
 'plans': 132,
 'bbc': 23,
 'afghanistan': 6,
 'uk': 179,
 'used': 183,
 'new': 114,
 'children': 35,
 'police': 134,
 'man': 103,
 'hit': 75,
 'turkey': 178,
 'crisis': 43,
 'world': 197,
 'report': 147,
 'china': 36,
 'israeli': 89,
 'peace': 130,
 'protest': 140,
 'years': 199,
 'oil': 122,
 'korean': 96,
 'protesters': 141,
 'want': 186,
 'soldiers': 162,
 'mexico': 106,
 'europe': 55,
 'national': 111,
 '000': 0,
 'men': 105,
 'day': 44,
 'united': 181,
 'states': 165,
 'human': 77,
 'rights': 149,
 'attack': 17,
 'iran': 84,
 'military': 107,
 'end': 53,
 'times': 175,
 'country': 41,
 'iraq': 86,
 'troops': 177,
 'billion': 24,
 'forces': 59,
 'government': 70,
 'al': 10,
 'war': 187,
 'time': 174,
 'free': 62,
 'saudi': 153,
 'po

In [116]:
text_features = pd.DataFrame(data=news_vectors,columns=vectorizer.get_feature_names())
text_features

,000,10,100,20,accused,afghan,afghanistan,africa,aid,air,...,weapons,week,west,wikileaks,woman,women,workers,world,year,years
0,0.087215,0.000000,0.000000,0.000000,0.000000,0.000000,0.119453,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.106088,0.000000,0.072767,0.000000,0.088168
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.097870,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.128819,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.068816,0.241812,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.145991,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.158046,0.092559,0.000000
4,0.000000,0.122114,0.000000,0.000000,0.000000,0.000000,0.119103,0.000000,0.290131,0.133461,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.105776,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105242,0.000000,0.128183,0.000000,...,0.000000,0.000000,0.113492,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077679
6,0.085786,0.000000,0.000000,0.000000,0.000000,0.000000,0.117496,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.313049,0.000000,0.071575,0.083835,0.086724
7,0.084017,0.000000,0.000000,0.000000,0.000000,0.000000,0.345221,0.000000,0.000000,0.000000,...,0.134224,0.000000,0.000000,0.00000,0.114908,0.000000,0.000000,0.210298,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.180586,0.000000,0.000000
9,0.096234,0.000000,0.000000,0.150034,0.000000,0.000000,0.131806,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.142138,0.00000,0.000000,0.000000,0.000000,0.321167,0.000000,0.000000


In [112]:
text_features = pd.concat([news.Date, text_features], axis=1)

In [128]:
pd.to_frame(news.Date.unique())

AttributeError: module 'pandas' has no attribute 'to_frame'

In [97]:
export = labels[['Date', 'Label']]

In [98]:
export.to_csv('interim_labels.csv')

In [102]:
data = text.merge(export, how='inner', left_on='Date', right_on='Date')

In [113]:
data

,Date,0,1,2,3,4,5,6,7,8,...,191,192,193,194,195,196,197,198,199,Label
0,2008-08-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101791,0.000000,0.000000,...,0.119172,0.000000,0.122756,0.000000,0.271205,0.233185,0.062008,0.072629,0.000000,0.0
1,2008-08-08,0.000000,0.000000,0.000000,0.000000,0.116894,0.000000,0.000000,0.000000,0.000000,...,0.119908,0.000000,0.000000,0.000000,0.272879,0.000000,0.000000,0.000000,0.000000,0.0
2,2008-08-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.196428,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053162,0.000000,0.000000,0.0
3,2008-08-08,0.087328,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.576971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,2008-08-08,0.164850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.136144,0.000000,0.000000,0.129308,0.068771,0.000000,0.000000,0.0
5,2008-08-08,0.000000,0.000000,0.000000,0.136800,0.000000,0.000000,0.000000,0.135251,0.000000,...,0.000000,0.000000,0.144931,0.000000,0.000000,0.000000,0.000000,0.085749,0.177408,0.0
6,2008-08-08,0.000000,0.000000,0.000000,0.000000,0.097704,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.206474,0.085482,0.000000,0.000000,0.104296,0.061081,0.126371,0.0
7,2008-08-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.154159,0.000000,0.0
8,2008-08-08,0.000000,0.101187,0.000000,0.224681,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.119018,0.000000,0.087649,0.000000,0.060120,0.070418,0.072844,0.0
9,2008-08-08,0.071574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.344309,0.105716,0.000000,0.097890,0.174124,0.000000,0.059717,0.139892,0.000000,0.0


In [105]:
vectorizer.get_feature()

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature'

array([[0.08721492, 0.        , 0.        , ..., 0.07276707, 0.        ,
        0.08816843],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.06881625, 0.24181161,
        0.        ],
       ...,
       [0.15848593, 0.        , 0.        , ..., 0.13223147, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07330361, 0.20587718, 0.        , ..., 0.06116028, 0.        ,
        0.14821006]])